In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base to prevent training
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = Flatten()(base_model.output)       # Flatten the output of the convolutional base
x = Dense(512, activation='relu')(x)   # Fully connected layer with 512 units and ReLU activation
x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation for binary classification

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation and rescaling
datagen = ImageDataGenerator(
    rescale=1./255,          # Rescale pixel values to [0, 1]
    validation_split=0.2     # Split 20% of data for validation
)

# Training data generator
train_generator = datagen.flow_from_directory(
    '/kaggle/input/stegoimagesdataset',  # Update with the actual path to your dataset
    target_size=(224, 224),               # Resize images to 224x224
    batch_size=32,                        # Batch size
    class_mode='binary',                  # Binary classification
    subset='training'                     # Use the 'training' subset
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    '/kaggle/input/stegoimagesdataset',  # Update with the actual path to your dataset
    target_size=(224, 224),               # Resize images to 224x224
    batch_size=32,                        # Batch size
    class_mode='binary',                  # Binary classification
    subset='validation'                  # Use the 'validation' subset
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,                           # Number of epochs
    validation_data=validation_generator
)

# Save the model if needed
model.save('/kaggle/working/vgg16_finetuned_model.h5')


Found 35200 images belonging to 3 classes.
Found 8800 images belonging to 3 classes.
Epoch 1/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 275s 247ms/step - accuracy: 0.5277 - loss: -12.4168 - val_accuracy: 0.5895 - val_loss: -136.7496
Epoch 2/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 270s 244ms/step - accuracy: 0.5805 - loss: -392.3898 - val_accuracy: 0.5731 - val_loss: -736.6566
Epoch 3/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 272s 246ms/step - accuracy: 0.5936 - loss: -1540.7452 - val_accuracy: 0.6595 - val_loss: -1931.7002
Epoch 4/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 267s 242ms/step - accuracy: 0.5956 - loss: -3452.9163 - val_accuracy: 0.5993 - val_loss: -3529.1934
Epoch 5/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 269s 243ms/step - accuracy: 0.5956 - loss: -5973.7441 - val_accuracy: 0.6828 - val_loss: -5413.2041
Epoch 6/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 270s 245ms/step - accuracy: 0.5974 - loss: -9188.8984 - val_accuracy: 0.6282 - val_loss: -7823.7910
Epoch 7/10
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 268s 243ms/step - accuracy

In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

def detect_steganographic_image(image_path, model_path):
    # Load the trained model
    model = load_model(model_path)
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Make a prediction
    prediction = model.predict(img_array)
    
    # Determine the class based on the prediction
    if prediction[0] > 0.5:
        print("The image is steganographic.")
    else:
        print("The image is normal.")

# Example usage with uploaded files
image_path = '/kaggle/working/your_uploaded_image.jpg'
model_path = '/kaggle/working/fine_tuned_model.h5'

detect_steganographic_image('/kaggle/input/sajal042/pngtree-normal-icon-png-image_6630696.jpg', '/kaggle/working/vgg16_finetuned_model.h5')
detect_steganographic_image('/kaggle/input/sajal2/cardad.jpg', '/kaggle/working/vgg16_finetuned_model.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
The image is normal.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
The image is steganographic.
